# Titanic: Machine Learning from Disaster

In [3]:
# https://www.kaggle.com/c/titanic

In [132]:
import sklearn
import pandas as pd

In [133]:
# Загружаем данные из файлов
train = pd.read_csv('./titanic/train.csv')
test = pd.read_csv('./titanic/test.csv')

In [134]:
# сохраняем столбец идентификаторов пассажиров для тестовых данных
test_passenger_id = test['PassengerId']

In [135]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Предобработка данных, добавление признаков

In [136]:
# https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

import re

RARE_TITLES = ['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']

# создаем словарь для исправления префиксов имени
TITLES = dict((title, 'Rare') for title in RARE_TITLES)

TITLES['Mlle'] = 'Miss'
TITLES['Ms'] = 'Miss'
TITLES['Mme'] = 'Mrs'

def get_title(name):
    """ Возвращает префикс имени """
    match = re.search(' ([A-Za-z]+)\.', name)
    if match:
        # исправляем опечатки в префиксе имени
        return TITLES.get(match.group(1), match.group(1))
    return ''

# Применяем процесс для обучающего и тестового наборов
for dataset in [test, train]:
    # длина имени
    dataset['Name_Length'] = train['Name'].apply(len)
    # была ли каюта у пассажира
    dataset['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
    # сколько членов семьи было на корабле
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    # флаг = 1, если путешествует в одиночестве
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    # заполняем пропуски для Embarked
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    # заполняем индексами
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    # заполняем пропуски Fare медианой
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
    # заполняем пропуски возраста медианой
    dataset['Age'] = dataset['Age'].fillna(train['Age'].median())
    # добавляем префикс имени как отдельный категориальный признак
    dataset['Title'] = dataset['Name'].apply(get_title)
    dataset['Title'] = dataset['Title'].map( {'Mr': 1, 
                                              'Miss': 2, 
                                              'Mrs': 3,
                                              'Master': 4,
                                              'Rare': 5} )
    # заполняем неизвестные префиксы нулями
    dataset['Title'] = dataset['Title'].fillna(0)
    # бинаризуем пол
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    # категоризируем стоимость билета
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    # категоризируем возраст
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age']

In [137]:
# Удаляем неиспользуемые столбцы
DROP_COL = ['PassengerId', 'Name', 'Ticket', 'Cabin']

X_train = train.drop(DROP_COL + ['Survived'], axis=1)
y_train = train['Survived']
X_test = test.drop(DROP_COL, axis=1)

## Предсказание моделей для стеккинга

In [138]:
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

def cross_val_predict_proba(estimator, X_train, y_train, random_state=None, n_splits=5):
    kfold = KFold(n_splits=n_splits, 
                  shuffle=True,
                  random_state=random_state)
    
    return cross_val_predict(estimator, 
                             X_train,
                             y_train,
                             cv=kfold, 
                             method='predict_proba')

# TODO: подобрать гиперпараметры отдельных моделей

# инициализирем модели с подобранными гиперпараметрами
estimators = [RandomForestClassifier(),
              ExtraTreesClassifier(),
              AdaBoostClassifier(),
              GradientBoostingClassifier()]

# получаем предсказания вероятностей ансамблей на кросс-валидации для обучающей выборки
train_pred = [cross_val_predict_proba(est, X_train, y_train) for est in estimators]
X_train_stack = np.stack([p[:, 1] for p in train_pred], axis=1)

# получаем предсказания ансамблей для тестовой выборки
test_pred = [est.fit(X_train, y_train).predict_proba(X_test) for est in estimators]
X_test_stack = np.stack([p[:, 1] for p in test_pred], axis=1)

## Объединяем предсказания ансамблей с помощью логистической регрессии

In [139]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

kfold = KFold(shuffle=True, n_splits=4, random_state=19746)
params = {'class_weight': ['balanced', None],
          'penalty': ['l1', 'l2'],
          'C': [0.4, 0.5, 0.6, 0.7]}

grid = GridSearchCV(LogisticRegression(), params, scoring='accuracy', cv=kfold)
grid.fit(X_train_stack, y_train)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')[:3]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_class_weight,param_penalty,params,rank_test_score,split0_test_score,...,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
7,0.001065,0.000329,0.823793,0.824172,0.5,None,l2,"{'C': 0.5, 'class_weight': None, 'penalty': 'l2'}",1,0.807175,...,0.816143,0.827844,0.811659,0.824850,0.86036,0.813154,0.000025,0.000077,0.021302,0.006704
11,0.001306,0.000340,0.823793,0.824172,0.6,None,l2,"{'C': 0.6, 'class_weight': None, 'penalty': 'l2'}",1,0.802691,...,0.816143,0.827844,0.816143,0.826347,0.86036,0.813154,0.000270,0.000073,0.021769,0.006449
15,0.001095,0.000290,0.822671,0.824546,0.7,None,l2,"{'C': 0.7, 'class_weight': None, 'penalty': 'l2'}",3,0.802691,...,0.811659,0.827844,0.816143,0.826347,0.86036,0.813154,0.000063,0.000013,0.022245,0.006773


In [140]:
predicted = grid.best_estimator_.predict(X_test_stack)

## Формируем фалй для отправки

In [141]:
with open('submission.txt', 'w') as out:
    out.write('PassengerId,Survived\n')
    for passenger, y in zip(test_passenger_id, predicted):
        out.write('%s,%s\n' % (passenger, y))